# Task 2a: Text summarization with small files with Titan Text Premier


In this notebook, you ingest a small string of text directly into the Amazon Bedrock API (using the Titan Text model) and instruct it to summarize the input text. You can apply this approach to summarize call transcripts, meeting transcripts, books, articles, blog posts, and other relevant content when the input text length is within the context size limits of the model.

## Task 2a.1: Environment setup

In this task, you set up your environment.

In [8]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## Task 2a.2: Writing prompt with text to be summarized

In this task, you use a short passage of text with fewer tokens than the maximum length supported by the foundation model. As a sample input text for this lab, you use a paragraph from an [AWS blog post](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) announcing Amazon Bedrock.

The prompt starts with an instruction `Please provide a summary of the following text.`. 

In [9]:
prompt_data = """

Please provide a summary of the following text:

Results-oriented Full Stack Software Engineer with 6+ years of professional experience designing and developing reliable, 
robust, and highly available applications for the Engineering, E-Commerce, and Financial sectors. Skilled in building 
scalable, high-performance web applications using enterprise frameworks like Spring Boot, REST and Angular applying strong 
problem-solving and analytical skills. Proficient in Java, JPA, Hibernate, Kafka, MySQL, and PostgreSQL, with hands-on 
experience in React and jQuery.
"""

## Task 2a.3: Creating request body with prompt and inference parameters 

In this task, you create the request body with the above prompt and inference parameters.

In [10]:
# request body
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":2048,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

## Task 2a.4: Invoke foundation model via Boto3

In this task, you send an API request to Amazon Bedrock specifying the request parameters: `modelId`, `accept`, and `contentType`. Following the provided prompt, the foundation model in Amazon Bedrock then summarizes the input text.

### Complete Output Generation

By default, the Amazon Bedrock service generates the entire summary for a given prompt in a single output. This can be slow if the model output contains many tokens. 

In [4]:
#model configuration and invoke the model
modelId = 'amazon.titan-text-premier-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

print(outputText)

AWS has announced Amazon Bedrock, a new service that provides access to foundation models (FMs) from AI21 Labs, Anthropic, Stability AI, and Amazon via an API. Bedrock aims to make it easy for customers to build and scale generative AI-based applications using FMs, democratizing access for all builders. The service offers a range of powerful FMs for text and images, including Amazon's Titan FMs, which consist of two new large language models (LLMs) that AWS is also announcing today. Bedrock's serverless experience allows customers to easily find the right model for their needs, customize FMs with their own data, and integrate and deploy them into their applications using familiar AWS tools and capabilities, without having to manage any infrastructure.


### Streaming Output Generation

Next, you explore how to use Amazon Bedrock's invoke_model_with_response_stream API to stream model outputs so users can consume outputs as they are generated. Rather than generating the full output at once, this API returns a ResponseStream that sends smaller output chunks from the model as they are produced. You can display these streaming outputs in a continuous, consumable view.

In [5]:
#invoke model with response stream
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

[{'chunk': {'bytes': b'{"outputText":"AWS","index":0}'}},
 {'chunk': {'bytes': b'{"outputText":" has","index":1}'}},
 {'chunk': {'bytes': b'{"outputText":" announced","index":2}'}},
 {'chunk': {'bytes': b'{"outputText":" Amazon","index":3}'}},
 {'chunk': {'bytes': b'{"outputText":" Bed","index":4}'}},
 {'chunk': {'bytes': b'{"outputText":"rock","index":5}'}},
 {'chunk': {'bytes': b'{"outputText":",","index":6}'}},
 {'chunk': {'bytes': b'{"outputText":" a","index":7}'}},
 {'chunk': {'bytes': b'{"outputText":" new","index":8}'}},
 {'chunk': {'bytes': b'{"outputText":" service","index":9}'}},
 {'chunk': {'bytes': b'{"outputText":" that","index":10}'}},
 {'chunk': {'bytes': b'{"outputText":" provides","index":11}'}},
 {'chunk': {'bytes': b'{"outputText":" access","index":12}'}},
 {'chunk': {'bytes': b'{"outputText":" to","index":13}'}},
 {'chunk': {'bytes': b'{"outputText":" foundation","index":14}'}},
 {'chunk': {'bytes': b'{"outputText":" models","index":15}'}},
 {'chunk': {'bytes': b'{"

In [6]:
from IPython.display import display_markdown,Markdown,clear_output

In [7]:
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

AWS has announced Amazon Bedrock, a new service that provides access to foundation models (FMs) from AI21 Labs, Anthropic, Stability AI, and Amazon via an API. Bedrock aims to make it easy for customers to build and scale generative AI-based applications using FMs, democratizing access for all builders. The service offers a range of powerful FMs for text and images, including Amazon's Titan FMs, which consist of two new large language models (LLMs) announced today. Bedrock's serverless experience allows customers to easily find the right model, customize FMs with their own data, and integrate and deploy them into their applications using familiar AWS tools and capabilities, without managing any infrastructure.

You have now experimented with using the boto3 SDK to access the Amazon Bedrock API. This SDK provides basic programmatic access to Bedrock capabilities. By leveraging this API, you were able to implement two use cases: 1) Generating an entire text summary of AWS news content at once, and 2) Streaming the summary output in chunks for incremental processing.

### Try it yourself
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file and continue with **Task2b.ipynb**.